<a href="https://colab.research.google.com/github/KatBotkowska/CodersCrew/blob/master/Zadanie_4_1_grupa_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Pobranie danych z portalu Indeed.com


Celem zadania  jest pobranie danych z portalu internetowego publikującego oferty pracy: Indeed.com . Indeed jest firmą z branży rekrutacyjnej, która na swoim portalu umożliwia publikowanie ofert pracy przez pracodawców, a dla poszukujących pracy proponuje m.in. wyszukiwarkę ofert oraz dane na temat potencjalnych pracodawców (wraz z opiniamii na ich temat).  

Zadanie ma na celu stworzenie baz zawierających dane na temat ofert pracy  w poniższych tematach:

1. Oferty pracy z zakresu data science
2. Oferty pracy dla języków programowania: java,  python, java script, c#, android (języki wybrano biorąc pod uwagę najbardziej popularne języki, wynikające z analizy GitHub i StackOverflow we wcześniejszych zadaniach)
3. Oferty pracy dla testerów automatycznych i manualnych


W tym zadaniu porównano oferty pracy dla polskiej wersji strony. Kluczem wyszukiwania była m.in. lokalizacja - skupiono się na miastach: Wrocław, Poznań, Kraków, Warszawa.

Dane wybierano z ogłoszeń dla podanych wyżej zapytań w następujacych zakresach:

* 'Title' - nazwa stanowiska
* 'Company' - nazwa firmy
* 'Location' - miasto
* 'Salary' - wynagrodzenie
* 'Summary' - krótki opis oferty
* 'Data' -data publikacji ogłoszenia 
* 'ReviewsAmount' - ilość opinii o firmie
* 'Stars' - ocena firmy


**Wykonanie Grupa 5**

In [0]:
#importy
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time, datetime
from datetime import date
from google.colab import files

## Przygotowanie 

**Funkcja wyciągająca nazwę stanowiska**

In [0]:
def title(div):
    title = div.find('div', class_='title').a['title']
    return title

**Funkcja wyciągająca nazwę firmy**

In [0]:
def company_name(div):
    try:
        company_name = div.find('span', class_ = 'company').text.strip()
    except:
        company_name = 'Not found'
    return company_name

**Funkcja wyciągająca miasto**

In [0]:
def location(div):
    location = div.find('span', attrs = {'class': 'location'}).text
    return location

**Funkcja wyciągająca wynagrodzenie**

*W dużej ilości ofert w Polsce brak jest danych na temat wynagrodzenia, dlatego w przypadku braku takich danych wskazywane jest "nothing found"*

In [0]:
def salary(div):
    try:
        salary = div.find('span', class_= 'salary no-wrap').text
    except:
        salary = 'nothing found'
    return salary

**Funkcja wyciągająca podsumowanie oferty - krótki opis stanowiska**

In [0]:
def summary(div):
    summary = div.find('div', class_ = 'summary').text.strip()
    return summary

**Funkcja wyciągająca datę publikacji ogłoszenia**

*W przypadku starszych ogłoszeń niż 30 dni, wskazywane jest "older than 1month"*

In [0]:
def publishing_date(div):
    ago = div.find('span', class_ = 'date').text.split()[0]
    if ago == '30+':
        publishing_date = 'older than 1month'
    elif ago == 'wczoraj':
        publishing_date = date.today() - datetime.timedelta(1)
    else:
        publishing_date = date.today() - datetime.timedelta(int(ago))
            
    return publishing_date

**Funkcja wyciągajaca ilość opinii o firmie**

*W przypadku braku opinii wskazywane jest "not reviewed yet"*

In [0]:
def reviews_amount(div):
    try:
        reviews_amount = div.find('span', class_ = 'slNoUnderline').text.replace(' opinii', '').replace('\xa0', '')
    except:
        reviews_amount ='Not reviewed yet'
    return reviews_amount

**Funkcja wyciągająca ocenę firmy na podstawie opinii użytkowników portalu**

*W przypadku braku opinii wskazywane jest "not reviewed yet"*

In [0]:
def extract_stars(div):
    try:
        company = div.find('span', class_ = 'company').text.strip().split()
        new_company = '-'.join(company)
        page_company = requests.get(f'https://pl.indeed.com/cmp/{new_company}').text
        soup_company = BeautifulSoup(page_company, 'html.parser')
        header_company = soup_company.find('span', class_ = 'cmp-CompanyRatings-value').text
        stars = float(header_company.replace(',', '.'))
    except:
        stars = 'Not review yet'
    return stars

**Funkcja zmieniajaca wyszukiwane słowa kluczowe na zapis właściwy dla docelowej strony internetowej z danymi wyszukiwania**

In [0]:
def changed_key_words(key_words):
    new_key = []
    for word in key_words:
        word1 = '+'.join(word.split())
        new_key.append(word1)
    return new_key

**Funkcja wyciągająca dane ze strony i podstron dla zadanej "głębokości" wyszukiwania i zadanych słów kluczowych**

In [0]:
def job_offerts(key_words):
    job_posts =[]
    for word in changed_key_words(key_words):
        for city in changed_key_words(cities):
            for start in range(0, max_results, 10):
                page = requests.get(f'https://pl.indeed.com/praca?q={word}&l={city}&start={start}')
                time.sleep(1)
                soup_scrap = BeautifulSoup(page.text,'html.parser')
                for div in soup_scrap.find_all('div', class_ = 'row'):
                    post = []
                    post.append(title(div)) #tytuł ogłoszenia
                    post.append(company_name(div))#nazwa firmy
                    post.append(location(div)) #lokalizacja
                    post.append(salary(div)) #salary
                    post.append(summary(div)) #podsumowanie
                    post.append(publishing_date(div)) #data publikacji
                    post.append(reviews_amount(div))#ilość opinii
                    post.append(extract_stars(div))#ilość gwiazdek
                    job_posts.append(post)
    return job_posts

**Funkcja zamieniajaca wyciągnięte dane na DATA FRAME**

In [0]:
def data_to_df(data):
    df = pd.DataFrame(data)
    df.columns = ['Title', 'Company', 'Location', 'Salary', 'Summary', 'Date', 'ReviewsAmount', 'Stars']
    return df

**Funkcja zmieniająca Data Frame na plik .csv do ściągnięcia z menu FIles Google Colab**

In [0]:
def df_to_csv(df):
    file = df.to_csv('dane.csv')
    return file

## Założenia główne: maksymalna ilość zagłębień, miasta

In [0]:
max_results = 101
cities = ['Wrocław', 'Kraków','Warszawa', 'Poznań']

## 1.  Oferty pracy z zakresu data science

In [0]:
key_words_ds = ['data', 'data scientist', 'data science', 'data scientists', 'analityk danych', 'data analyst']

In [0]:
data_sc_offerts = job_offerts(key_words_ds)

In [0]:
df_ds = data_to_df(data_sc_offerts)
df_ds

,Title,Company,Location,Salary,Summary,Date,ReviewsAmount,Stars
0,Data Integrity & UAT Projects Analyst,Credit Suisse,"Wrocław, dolnośląskie",nothing found,Data Integrity & UAT Projects Analyst # 134837...,2019-07-05,1363,3.9
1,Data Analyst,Hewlett Packard Enterprise,"Wrocław, dolnośląskie",nothing found,"Good analytical skills, knowledge of and exper...",2019-07-02,3984,3.7
2,Data Analyst - Business Intelligence,XL Catlin,"Wrocław, dolnośląskie",nothing found,The Senior Data Analyst will be focused on per...,older than 1month,90,4
3,(Junior) Data Analyst in Conduct Risk Team,Credit Suisse,"Wrocław, dolnośląskie",nothing found,(Junior) Data Analyst in Conduct Risk Team # 1...,2019-06-28,1363,3.9
4,Data Scientist,EY,"Wrocław, dolnośląskie",nothing found,Processing huge amount of data and extracting ...,2019-06-28,6202,4
5,Data Coordinator FP&I,Pattonair,"Wrocław, dolnośląskie",nothing found,Data Coordinator FP&I. As a Data Coordinator y...,2019-06-07,Not reviewed yet,2.8
6,Data Analyst,Credit Suisse,"Wrocław, dolnośląskie",nothing found,Data Analyst # 131565. Knowledge of data visua...,older than 1month,1363,3.9
7,"Data Analyst - Business Intelligence, MicroStr...",XL Catlin,"Wrocław, dolnośląskie",nothing found,Passion for data and experience working within...,older than 1month,90,4
8,Graduate Opportunities: Customer Master Data S...,Becton Dickinson,"Wrocław, dolnośląskie",nothing found,Master Data Specialist participates in buildin...,older than 1month,2107,3.9
9,Instrument Reference Data Specialist,Credit Suisse,"Wrocław, dolnośląskie",nothing found,Instrument Reference Data Specialist. You will...,older than 1month,1363,3.9


In [0]:
file = 'data_science.csv'
df_ds.to_csv(file)

## 2. Oferty pracy dla języków programowania: java,  python, java script, c#, android

In [0]:
key_words_lang = ['java', 'python', 'java script', 'c#', 'android']

In [0]:
lang_offerts = job_offerts(key_words_lang)

In [0]:
df_lang = data_to_df(lang_offerts)
df_lang

,Title,Company,Location,Salary,Summary,Date,ReviewsAmount,Stars
0,IT Developer – Java,3M,"Wrocław, dolnośląskie",nothing found,Should be strong in Java basics/fundamental co...,2019-07-05,3385,4.1
1,Junior Java Developer,Cryptomage,"Wrocław, dolnośląskie",nothing found,Java skills. Junior Java Developer. BSc or MSc...,older than 1month,Not reviewed yet,Not review yet
2,Java Working Student,Nokia Wrocław,"Wrocław, dolnośląskie",nothing found,"Java EE, JavaScript, AngularJS, NodeJS. Knowle...",older than 1month,4249,4.1
3,Java Developer - FX Trading,UBS,"Wrocław, dolnośląskie",nothing found,Strong Java development skills (currently on J...,2019-06-28,1854,3.9
4,Data Scientist,EY,"Wrocław, dolnośląskie",nothing found,About EY GDS:. EY Global Delivery Services mea...,2019-06-28,6202,4
5,Młodszy Analityk Systemów Informatycznych,NETIA,"Wrocław, dolnośląskie",nothing found,"Znajomość języka Java 8,. Netia jest jednym z ...",2019-07-05,6,4
6,Java Software Engineer Intern,UBS,"Wrocław, dolnośląskie",nothing found,Experience with Java and the Spring Framework ...,2019-06-13,1854,3.9
7,Intern,Cryptomage,"Wrocław, dolnośląskie",nothing found,Internship Requirements: Education Student Jav...,older than 1month,Not reviewed yet,Not review yet
8,Full Stack Java Developer,UBS,"Wrocław, dolnośląskie",nothing found,"Proficiency in core Java programming, Spring F...",2019-07-05,1854,3.9
9,Java Developer - GALM IT Optimisation,UBS,"Wrocław, dolnośląskie",nothing found,Expertise in Java 8 with knowledge of low-leve...,2019-07-01,1854,3.9


In [0]:
file = 'lang.csv'
df_lang.to_csv(file)

## 3. Oferty pracy dla testerów automatycznych i manualnych

In [0]:
key_words_tests = ['tester automatyczny', 'tester manualny']

In [0]:
tests_offerts = job_offerts(key_words_tests)

In [0]:
df_tests = data_to_df(tests_offerts)
df_tests

,Title,Company,Location,Salary,Summary,Date,ReviewsAmount,Stars
0,Tester automatyczny,Droptica,"Wrocław, dolnośląskie",nothing found,"Miejsce: Wrocław, Gdańsk Zobacz szczegóły > ...",older than 1month,Not reviewed yet,5
1,Tester Automatyczny - Wrocław,Sii Polska,"Wrocław, dolnośląskie",nothing found,Tester oprogramowania - Wyzwania:. Minimum kil...,older than 1month,Not reviewed yet,5
2,Tester automatyczny ze znajomością C# / Automa...,Sii Polska,"Wrocław, dolnośląskie",nothing found,Minimum 2 lata doświadczenia w pracy na stanow...,older than 1month,Not reviewed yet,5
3,Tester automatyczny (Gdańsk/Wrocław),Droptica,"Wrocław, dolnośląskie",nothing found,Doświadcznie na stanowisku testera. Dołączenie...,older than 1month,Not reviewed yet,5
4,Customer Service Specialist (French),Comarch,"Wrocław, dolnośląskie",nothing found,"Wsparcia wdrożeń, Tester, Analityk). Reagowani...",2019-06-14,Not reviewed yet,4
5,Tester automatyczny,Droptica,"Wrocław, dolnośląskie",nothing found,"Miejsce: Wrocław, Gdańsk Zobacz szczegóły > ...",older than 1month,Not reviewed yet,5
6,Tester Automatyczny - Wrocław,Sii Polska,"Wrocław, dolnośląskie",nothing found,Tester oprogramowania - Wyzwania:. Minimum kil...,older than 1month,Not reviewed yet,5
7,Tester automatyczny ze znajomością C# / Automa...,Sii Polska,"Wrocław, dolnośląskie",nothing found,Minimum 2 lata doświadczenia w pracy na stanow...,older than 1month,Not reviewed yet,5
8,Tester automatyczny (Gdańsk/Wrocław),Droptica,"Wrocław, dolnośląskie",nothing found,Doświadcznie na stanowisku testera. Dołączenie...,older than 1month,Not reviewed yet,5
9,Customer Service Specialist (French),Comarch,"Wrocław, dolnośląskie",nothing found,"Wsparcia wdrożeń, Tester, Analityk). Reagowani...",2019-06-14,Not reviewed yet,4


In [0]:
file = 'tests.csv'
df_tests.to_csv(file)